## Imports

In [2]:
# necessary imports
import requests
import re # regex

## Inspection

Like most web scraping ventures, I start by seeing if I could get the contents of the `html` file first. I begin my intial dive with [UC Berkeley's Spring 2023 Semester's CS61A's Academic Interns (AIs)](https://cs61a.org/academic-interns/).

In [4]:
# Scrape the html contents
url = "https://cs61a.org/academic-interns/"
# API call
response = requests.get(url)
# Store response into a string
response_string = response.text

print(response_string[:100])


<!DOCTYPE html>
<html lang="en">
  <head>
    <meta name="description" content ="CS 61A: Structure 


Unfortunately, unlike [my other web scraping project](https://github.com/dnhuang/sf-tri-city-restaurants/blob/main/scraper.ipynb), there doesn't seem to to be a JSON response file shown by Google Chrome developer tools. This means that I needed to do this the old fashioned way: regular expressions.

Thankfully, online tools like [Regex101](https://regex101.com/) exists, which made finding a pattern much easier. But in the process of figuring out patterns, I noticed a lot of inconsistencies and edge cases that were very difficult to capture; all from **ONE** semester of 61A. Here are a few of them:

#### **Names**

I was an AI for 61A in Fall 2021, and I only recall inputting my first and last name, so I thought the pattern would be pretty simple. It turns out some people also listed their middle names, which meant there were at least two spaces. But then came the Indian names, which maybe had like five different words strung together. After accounting for these, I was still was getting the correct amount of matches (there are a total of $130$ AIs for Spring 2023 and yes, I manually counted them). So that's when I discovered that some people had hyphens (-) in their name. After adding that into the pattern, I was still left disappointed; $129/130$ matches. And that's when I saw this:

<div>
<img src='img/parentheses_in_name.png' width=500>
</div>

Of course, ***nicknames***.

After accounting for all of these edge cases, I arrived at the following pattern: `([a-zA-Z\-\(\) ]+)`

It may seem simple at first, but this was after spending quite some time refining it from the crazy mess I had intially, and to be honest I'm quite proud of it.

Thinking back on it now, the question that I'm trying to answer really only needs pronouns and the corresponding person's bio... name really isn't needed. But since I've already spent so much time figuring out a regex pattern to capture a person's full name, I thought I might as well scrape it.

Moving forward, I will no longer be account for any other edge cases or weird things in future scrapes unless I get some really broken data. I hope this statement ages well as I move through this project.

#### **Pronouns**

Pronouns weren't as bad as names, but that might be because I didn't bother spending time to make the pattern look concise; I'm kind of just over it. An edge case I ran into was this:

<div>
<img src='img/multi_pronoun.png' width=500>
<img src='img/multi_pronoun_html.png' width=500>
</div>

And this was the reason I was getting $131$ matches instead of $130$. ***Multi-set pronouns***. WHY.

I decided that I honestly couldn't be bothered to determine how to settle this issue, so I settled on just choosing the first set, as my pattern was only capturing the first set and I didn't want to fuss with it any further. With all that said, I arrived at the follwoing pattern for pronouns: `'<div class="badge badge-info">([a-zA-Z]*\/*[a-zA-Z]*\/*[a-zA-Z]*\/*)<\/div>'`

Yes, definitely not the prettiest. But like I said, I [CBA](https://www.urbandictionary.com/define.php?term=cba).

#### **Bio**

This was by far the worst one to extract. In the beginning, I thought I solved it by just simply using `(.*)` as I ended up $124$ matches (missing $6%) so I just needed to tweak it a little bit further.

But then I started to really struggle because I couldn't for the life of me figure out how to use `.*` within a `[]`, as I wanted to capture everything while account for `\s*`, which needed to be utilized within a `[]`.

But from my Google searches, it doesn't seem like `[.*]` is a valid regex pattern. I tried using `[\S\s]*` but this captured everything, like *literally everything* so I was out of luck there. But why do I care so much about the `\s`? Well, here is the reason:

<div>
<img src='img/multiline_bio_html.png'>
</div>

For some reason, some bios were not contained in *one line*. There is a newline character in this bio, which required me to somehow capture it in my regex pattern. After a few minutes of trial and error, I gave up and essentially did the regex version of hardcoding: `'<li class="section bio">(.{0,500}\s?.{0,500}\s?.{0,500}\s?.{0,500}\s?)<\/li>'`

In layman's terms, what this pattern is doing is capturing $500$ character chunks at a time not including newlines. Then it *checks* whether or not a newline character `\s` exists, then moves on to check another chunk. For Spring 2023 CS61A AIs, it works, so I'm done. Maybe I'll come around to making it look pretty in the future, but until then, this will do.

## Scraping

#### Initial Scraping of Spring 2023 CS61A AIs

With those aforementioned edge cases, I try my hand 

In [ ]:
### DEPRECATED CODE ###
# This was code that I intially wrote for extracting names and pronouns separately. At the end
# I decided to just combine the two patterns, looks neater in my opinion so I kept it.
# Functionality wise I don't think this changes much; intead of accessing two arrays containing singular 
# elements I would just be using a single array containing a list with two elements.

# # Extract name
# name_pattern = '<h3 class="staff-name">[\s\n]*([a-zA-Z ]+)\n'
# name_list = re.findall(name_pattern, response_string)
# print(name_list)


# # Extract pronouns
# pronoun_pattern = '<div class="badge badge-info">([a-zA-Z]*\/*[a-zA-Z]*\/*[a-zA-Z]*\/*)<\/div>'
# pronoun_list = re.findall(pronoun_pattern, response_string)
# print(pronoun_list)

In [ ]:
### DEPRECATED, decided to scrape as a pair ###


# Extract name and pronouns
name_pronoun_pattern = '<h3 class="staff-name">[\s\n]*([a-zA-Z\-\(\) ]+)\s+<div class="badge badge-info">([a-zA-Z]*\/*[a-zA-Z]*\/*[a-zA-Z]*\/*)<\/div>'
name_pronoun_list = re.findall(name_pronoun_pattern, response_string)
print(name_pronoun_list)

# Extract bio
bio_pattern = '<li class="section bio">(.{0,500}\s?.{0,500}\s?.{0,500}\s?.{0,500}\s?)<\/li>'
bio_list = re.findall(bio_pattern, response_string)


In [15]:
print(len(pronoun_list))

131


In [16]:
print(len(name_list))

130


In [27]:
print(len(bio_list))
print(len(name_pronoun_list))

130
130
